<a href="https://colab.research.google.com/github/Sebagi-community/Workshop/blob/main/workshop/rasa/app.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 파이썬을 활용한 챗봇 실행파일


In [32]:
!pip install sounddevice numpy SpeechRecognition beautifulsoup4

In [33]:
!pip install sounddevice soundfile SpeechRecognition

In [34]:
!pip install numpy

In [35]:
!pip install sounddevice numpy SpeechRecognition

In [31]:
!pip install --upgrade pip

  Using cached pip-24.2-py3-none-any.whl.metadata (3.6 kB)
Using cached pip-24.2-py3-none-any.whl (1.8 MB)
  Attempting uninstall: pip
    Found existing installation: pip 24.1.2
    Uninstalling pip-24.1.2:
      Successfully uninstalled pip-24.1.2


In [46]:
!!pip install flask

['Requirement already satisfied: flask in /usr/local/lib/python3.10/dist-packages (2.2.5)',
 'Requirement already satisfied: Werkzeug>=2.2.2 in /usr/local/lib/python3.10/dist-packages (from flask) (3.0.4)',
 'Requirement already satisfied: Jinja2>=3.0 in /usr/local/lib/python3.10/dist-packages (from flask) (3.1.4)',
 'Requirement already satisfied: itsdangerous>=2.0 in /usr/local/lib/python3.10/dist-packages (from flask) (2.2.0)',
 'Requirement already satisfied: click>=8.0 in /usr/local/lib/python3.10/dist-packages (from flask) (8.1.7)',
 'Requirement already satisfied: MarkupSafe>=2.0 in /usr/local/lib/python3.10/dist-packages (from Jinja2>=3.0->flask) (2.1.5)']

In [47]:
!pip install --upgrade flask-cors


  Attempting uninstall: flask-cors
    Found existing installation: Flask-Cors 1.10.1
    Uninstalling Flask-Cors-1.10.1:
      Successfully uninstalled Flask-Cors-1.10.1


In [37]:
!pip install requests beautifulsoup4

In [38]:
!pip install flask-cors

In [2]:
!pip install Flask flask-cors SpeechRecognition

In [39]:
!pip install gunicorn

In [40]:
!gunicorn --workers 3 --bind 0.0.0.0:5005 main:app

[2024-09-20 09:19:33 +0000] [8825] [INFO] Starting gunicorn 23.0.0
[2024-09-20 09:19:33 +0000] [8825] [INFO] Listening at: http://0.0.0.0:5005 (8825)
[2024-09-20 09:19:33 +0000] [8825] [INFO] Using worker: sync
[2024-09-20 09:19:33 +0000] [8826] [INFO] Booting worker with pid: 8826
[2024-09-20 09:19:33 +0000] [8826] [ERROR] Exception in worker process
Traceback (most recent call last):
  File "/usr/local/lib/python3.10/dist-packages/gunicorn/arbiter.py", line 608, in spawn_worker
    worker.init_process()
  File "/usr/local/lib/python3.10/dist-packages/gunicorn/workers/base.py", line 135, in init_process
    self.load_wsgi()
  File "/usr/local/lib/python3.10/dist-packages/gunicorn/workers/base.py", line 147, in load_wsgi
    self.wsgi = self.app.wsgi()
  File "/usr/local/lib/python3.10/dist-packages/gunicorn/app/base.py", line 66, in wsgi
    self.callable = self.load()
  File "/usr/local/lib/python3.10/dist-packages/gunicorn/app/wsgiapp.py", line 57, in load
    return self.load_wsgia

In [41]:
!pip install flask-cors==1.10.1

In [42]:
!pip install SpeechRecognition

In [43]:
!pip install python-dotenv

In [ ]:
from flask import Flask, render_template, request, jsonify
from flask_cors import CORS
import sounddevice as sd
import numpy as np
import speech_recognition as sr
import io
import wave
import logging
from dotenv import load_dotenv
import os
import requests
from bs4 import BeautifulSoup


app = Flask(__name__)
CORS(app)

# 로깅 설정
logging.basicConfig(level=logging.DEBUG)

# .env 파일에서 환경 변수 로드
load_dotenv()

# 음성 녹음 함수
def record_audio(duration=5, samplerate=16000):  # 샘플링 속도 16000Hz로 변경
    logging.info(f"{duration}초 동안 녹음을 시작합니다...")
    try:
        recording = sd.rec(int(duration * samplerate), samplerate=samplerate, channels=1, dtype='int16')
        sd.wait()
        with io.BytesIO() as wav_buffer:
            with wave.open(wav_buffer, 'wb') as wav_file:
                wav_file.setnchannels(1)
                wav_file.setsampwidth(2)
                wav_file.setframerate(samplerate)
                wav_file.writeframes(recording.tobytes())
            wav_buffer.seek(0)
            logging.info("녹음이 완료되었습니다!")
            return wav_buffer.read()
    except Exception as e:
        logging.error(f"녹음 중 에러 발생: {e}")
        return None

# 음성을 텍스트로 변환하는 함수
def transcribe_audio(audio_data):
    recognizer = sr.Recognizer()
    try:
        audio = sr.AudioData(audio_data, 16000, 2)  # 샘플링 속도 16000Hz로 변경
        text = recognizer.recognize_google(audio, language='ko-KR')
        logging.info(f"인식된 텍스트: {text}")
        return text
    except sr.UnknownValueError:
        logging.error("음성을 인식할 수 없습니다.")
        return None
    except sr.RequestError as e:
        logging.error(f"Google Speech Recognition 서비스 에러: {e}")
        return None
    except Exception as e:
        logging.error(f"알 수 없는 오류 발생: {e}")
        return None

# 네이버 뉴스 제목 가져오는 함수
def get_news_titles(query, num_results=5):
    url = f"https://search.naver.com/search.naver?where=news&query={requests.utils.quote(query)}"  # 쿼리 인코딩
    headers = {
        'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/91.0.4472.124 Safari/537.36'
    }

    try:
        response = requests.get(url, headers=headers)
        response.raise_for_status()
        soup = BeautifulSoup(response.text, 'html.parser')

        titles_and_links = []
        for a_tag in soup.find_all('a', class_='news_tit', limit=num_results):
            title = a_tag.get_text()
            link = a_tag['href']
            titles_and_links.append({'title': title, 'link': link})

        return titles_and_links
    except requests.RequestException as e:
        logging.error(f"뉴스 제목 가져오기 중 에러 발생: {e}")
        return []

# 기존 라우트
@app.route('/')
def home():
    return render_template('index.html')

@app.route('/data', methods=['GET'])
def get_data():
    return jsonify({'message': 'Data endpoint'})

@app.route('/new', methods=['GET'])
def new_feature():
    return jsonify({'message': 'This is a new feature'})

@app.route('/speech-to-text', methods=['POST'])
def speech_to_text():
    try:
        audio_data = record_audio(duration=3)  # 3초 동안 녹음
        if audio_data is None:
            logging.error("녹음에 실패했습니다.")
            return jsonify({'message': "녹음에 실패했습니다. 서버 로그를 확인하십시오."}), 400

        text = transcribe_audio(audio_data)
        if text:
            titles = get_news_titles(text, num_results=10)
            response = {
                'text': text,
                'titles': titles
            }
            return jsonify(response), 200
        else:
            logging.error("음성 인식에 실패했습니다.")
            return jsonify({'message': "음성 인식에 실패했습니다. 서버 로그를 확인하십시오."}), 500
    except Exception as e:
        logging.exception("서버 오류 발생")
        return jsonify({'message': "서버 오류가 발생했습니다. 서버 로그를 확인하십시오."}), 500

# if __name__ == '__main__':
#     app.run(host='localhost', port=5005, debug=False)


if __name__ == '__main__':
    app.run()


 * Serving Flask app '__main__'
 * Debug mode: off


INFO:werkzeug:WARNING: This is a development server. Do not use it in a production deployment. Use a production WSGI server instead.
 * Running on http://127.0.0.1:5000
INFO:werkzeug:Press CTRL+C to quit
